In [6]:
from bs4 import BeautifulSoup
from dateutil.parser import parse
from datetime import datetime
from datetime import datetime, timedelta
import urlparse
import pymongo 
from pymongo import MongoClient
import os

In [7]:
connection = MongoClient()
db = connection['espn_leagues_drafts']
auction_drafts = db['auction_drafts']
league_settings = db['league_settings']

In [8]:
print 'auction draft picks:',auction_drafts.count()
auction_drafts.find_one()

auction draft picks: 2370380


{u'_id': ObjectId('5696f378c807ee40f02d507e'),
 u'league_id': u'100005',
 u'nom_order': 16,
 u'player_id': u'4529',
 u'player_name': u'Yu Darvish',
 u'player_name_id': u'Yu Darvish (4529)',
 u'price': 1,
 u'season_id': u'2015',
 u'team_id': u'1',
 u'team_name': u'LEAGUE CHAMPION',
 u'unique_team_id': u'-2147483648'}

In [9]:
print 'auction drafts collected/stored:', len(auction_drafts.distinct("league_id"))

auction drafts collected/stored: 9359


In [10]:
league_settings.find_one()

{u'2015 Keeper Lock Date': u'Lock One Hour Before Draft',
 u'2015 Keepers Per Team': u'None',
 u'2016 Keeper Lock Date': u'Lock One Hour Before Draft',
 u'2016 Keepers Per Team': u'None',
 u'Allow Draft Pick Trading': u'Yes',
 u'Auction Budget': u'0',
 u'Current Roster Size': u' 20',
 u'Draft Date': u'Sunday, March 29, 2015 at 8:15 PM ET',
 u'Draft Order': u'Manually Set by League Manager',
 u'Draft Type': u'Snake',
 u'Format': u'League Manager',
 u'Keeper Note': u'',
 u'League Name': u"MLW (Malibu's Least Wanted)",
 u'Location': u'',
 u'Make League Viewable to Public': u'Yes',
 u'Note': u'',
 u'Number of Teams': u'12',
 u'Scoring Type': u'Head to Head Each Category',
 u'Seconds Per Pick': u'60',
 u'Total On Bench': u' 5 (2 DL)',
 u'Total Starters': u' 15',
 u'Use Keepers for 2015 Season': u'No',
 u'Use Keepers for 2016 Season': u'No',
 u'_id': ObjectId('5696ea45c807ee4d9977fd18'),
 u'league_id': u'10000',
 u'positions': {u'1B': u'1',
  u'2B': u'1',
  u'3B': u'1',
  u'BE': u'5',
  u'C'

In [11]:
league_settings.group(["Format"], {}, {"count":0},"function(o, p){p.count++}" )

[{u'Format': u'League Manager', u'count': 19409.0},
 {u'Format': u'Standard', u'count': 30421.0},
 {u'Format': u'ESPN Custom', u'count': 20320.0}]

In [12]:
league_settings.group(["Scoring Type"], {}, {"count":0},"function(o, p){p.count++}" )

[{u'Scoring Type': u'Head to Head Each Category', u'count': 23428.0},
 {u'Scoring Type': u'Rotisserie', u'count': 18799.0},
 {u'Scoring Type': u'Head to Head Points', u'count': 22658.0},
 {u'Scoring Type': u'Head to Head Most Categories', u'count': 3886.0},
 {u'Scoring Type': u'Total Season Points', u'count': 1379.0}]

In [13]:
league_settings.group(["Draft Type"], {}, {"count":0},"function(o, p){p.count++}" )

[{u'Draft Type': u'Snake', u'count': 54688.0},
 {u'Draft Type': u'Auction', u'count': 9359.0},
 {u'Draft Type': u'Autopick', u'count': 3666.0},
 {u'Draft Type': u'Offline', u'count': 2436.0},
 {u'Draft Type': None, u'count': 1.0}]

In [14]:
league_settings.group(["Number of Teams"], {}, {"count":0},"function(o, p){p.count++}" )

[{u'Number of Teams': u'12', u'count': 6715.0},
 {u'Number of Teams': u'10', u'count': 46598.0},
 {u'Number of Teams': u'8', u'count': 6926.0},
 {u'Number of Teams': u'14', u'count': 935.0},
 {u'Number of Teams': u'20', u'count': 694.0},
 {u'Number of Teams': u'6', u'count': 2941.0},
 {u'Number of Teams': u'16', u'count': 814.0},
 {u'Number of Teams': u'4', u'count': 3000.0},
 {u'Number of Teams': u'5', u'count': 359.0},
 {u'Number of Teams': u'9', u'count': 320.0},
 {u'Number of Teams': u'13', u'count': 95.0},
 {u'Number of Teams': u'15', u'count': 133.0},
 {u'Number of Teams': u'7', u'count': 258.0},
 {u'Number of Teams': u'11', u'count': 262.0},
 {u'Number of Teams': u'18', u'count': 87.0},
 {u'Number of Teams': u'17', u'count': 12.0},
 {u'Number of Teams': u'19', u'count': 1.0}]

In [149]:
league_settings.group(["Use Keepers for 2015 Season"], {}, {"count":0},"function(o, p){p.count++}" )

[{u'Use Keepers for 2015 Season': u'No', u'count': 17963.0},
 {u'Use Keepers for 2015 Season': u'Yes', u'count': 4543.0}]

In [16]:
auction_standard_10team_league_ids = list(league_settings.find({"Draft Type":"Auction","Number of Teams":"10",
                                                         "Format":"Standard",
               "Auction Budget":"260","Scoring Type":"Rotisserie",
               "Use Keepers for 2015 Season":"No"},{"_id":0,"league_id":1}))

auction_standard_10team_league_ids =[d['league_id'] for d in auction_standard_10team_league_ids]
print len(auction_standard_10team_league_ids)
#auction_standard_10team_league_ids

2478


In [17]:
auction_drafts = db['auction_drafts']
pipe = [{'$match':{'league_id':{'$in': auction_standard_10team_league_ids}}},
        { '$out' : "standard_auction_10team_roto_picks" }]

auction_drafts.aggregate(pipeline=pipe)
standard_auction_10team_roto_picks = db['standard_auction_10team_roto_picks']
standard_auction_10team_roto_picks.count()

619500

In [21]:
pipe = [{'$group':{'_id':'$player_name_id','total':{'$sum':1},'avg_price':{'$avg':'$price'}}},
        { '$sort' : { 'total':-1} },
        { '$out' : "auction_standard_10team_avg_prices" }
        ]

standard_auction_10team_roto_picks.aggregate(pipeline=pipe)

auction_standard_10team_avg_prices = db['auction_standard_10team_avg_prices']
auction_standard_10team_avg_prices.count()

1022

In [23]:
for player in auction_standard_10team_avg_prices.find()[:500]:
    print player

{u'total': 2478, u'_id': u'Gregory Polanco (5447)', u'avg_price': 6.094430992736077}
{u'total': 2478, u'_id': u'Craig Kimbrel (3056)', u'avg_price': 16.617029862792574}
{u'total': 2478, u'_id': u'Jacob deGrom (5295)', u'avg_price': 10.563357546408394}
{u'total': 2478, u'_id': u'Dellin Betances (3584)', u'avg_price': 8.21912832929782}
{u'total': 2478, u'_id': u'Kenley Jansen (2648)', u'avg_price': 9.740516545601292}
{u'total': 2478, u'_id': u'Felix Hernandez (1129)', u'avg_price': 33.8091202582728}
{u'total': 2478, u'_id': u'Sonny Gray (4555)', u'avg_price': 13.615415657788539}
{u'total': 2478, u'_id': u'Lance Lynn (3257)', u'avg_price': 6.384584342211461}
{u'total': 2478, u'_id': u'Gio Gonzalez (1878)', u'avg_price': 11.192090395480227}
{u'total': 2478, u'_id': u'Jonathan Lucroy (2816)', u'avg_price': 12.056900726392252}
{u'total': 2478, u'_id': u'Matt Holliday (997)', u'avg_price': 11.313962873284908}
{u'total': 2478, u'_id': u'Bryce Harper (3390)', u'avg_price': 19.526230831315576}
{

In [312]:
pipe = [{'$match':{'player_id':'3280'}},
        { '$out' : "mike_trout_standard_auction_10team_roto_picks" }]

standard_auction_10team_roto_picks.aggregate(pipeline=pipe)
mike_trout_standard_auction_10team_roto_picks = db['mike_trout_standard_auction_10team_roto_picks']
#mike_trout_standard_auction_10team_roto_picks.count()
pipe = [{'$group':{'_id':'$nom_order','total':{'$sum':1},'avg_price':{'$avg':'$price'}}},
        { '$sort' : { '_id':1} }
        ]
list(mike_trout_standard_auction_10team_roto_picks.aggregate(pipeline=pipe))

[{u'_id': 1, u'avg_price': 50.26676602086438, u'total': 671},
 {u'_id': 2, u'avg_price': 51.65814696485623, u'total': 313},
 {u'_id': 3, u'avg_price': 51.7, u'total': 160},
 {u'_id': 4, u'avg_price': 51.28260869565217, u'total': 92},
 {u'_id': 5, u'avg_price': 51.208955223880594, u'total': 67},
 {u'_id': 6, u'avg_price': 50.23684210526316, u'total': 38},
 {u'_id': 7, u'avg_price': 50.05555555555556, u'total': 36},
 {u'_id': 8, u'avg_price': 51.24, u'total': 25},
 {u'_id': 9, u'avg_price': 49.11764705882353, u'total': 17},
 {u'_id': 10, u'avg_price': 50.785714285714285, u'total': 14},
 {u'_id': 11, u'avg_price': 47.666666666666664, u'total': 3},
 {u'_id': 12, u'avg_price': 51.0, u'total': 4},
 {u'_id': 14, u'avg_price': 50.75, u'total': 4},
 {u'_id': 16, u'avg_price': 53.0, u'total': 2},
 {u'_id': 18, u'avg_price': 46.0, u'total': 1},
 {u'_id': 19, u'avg_price': 42.0, u'total': 1},
 {u'_id': 21, u'avg_price': 51.0, u'total': 2},
 {u'_id': 26, u'avg_price': 52.0, u'total': 1},
 {u'_id': 

In [315]:
pipe = [{'$match':{'player_id':'1870'}},
        { '$out' : "scherz_standard_auction_10team_roto_picks" }]

standard_auction_10team_roto_picks.aggregate(pipeline=pipe)
scherz_standard_auction_10team_roto_picks = db['scherz_standard_auction_10team_roto_picks']
#mike_trout_standard_auction_10team_roto_picks.count()
pipe = [{'$group':{'_id':'$nom_order','total':{'$sum':1},'avg_price':{'$avg':'$price'}}},
        { '$sort' : { '_id':1} }
        ]
list(scherz_standard_auction_10team_roto_picks.aggregate(pipeline=pipe))

[{u'_id': 1, u'avg_price': 25.8, u'total': 5},
 {u'_id': 2, u'avg_price': 23.0, u'total': 2},
 {u'_id': 3, u'avg_price': 28.4, u'total': 5},
 {u'_id': 4, u'avg_price': 25.625, u'total': 8},
 {u'_id': 5, u'avg_price': 27.8, u'total': 10},
 {u'_id': 6, u'avg_price': 24.22222222222222, u'total': 9},
 {u'_id': 7, u'avg_price': 29.11111111111111, u'total': 9},
 {u'_id': 8, u'avg_price': 26.142857142857142, u'total': 7},
 {u'_id': 9, u'avg_price': 28.0, u'total': 14},
 {u'_id': 10, u'avg_price': 25.923076923076923, u'total': 13},
 {u'_id': 11, u'avg_price': 27.470588235294116, u'total': 17},
 {u'_id': 12, u'avg_price': 27.1875, u'total': 16},
 {u'_id': 13, u'avg_price': 25.571428571428573, u'total': 28},
 {u'_id': 14, u'avg_price': 26.5, u'total': 16},
 {u'_id': 15, u'avg_price': 26.782608695652176, u'total': 23},
 {u'_id': 16, u'avg_price': 26.88235294117647, u'total': 17},
 {u'_id': 17, u'avg_price': 26.36, u'total': 25},
 {u'_id': 18, u'avg_price': 25.8, u'total': 25},
 {u'_id': 19, u'avg